In [1]:
# !pip install langchain qdrant-client fastapi bcrypt jwt pyjwt python-jose sentence-transformers ctransformers passlib itsdangerous

In [2]:
from fastapi import APIRouter, Request, Depends, status
from fastapi.responses import HTMLResponse, RedirectResponse
from fastapi.templating import Jinja2Templates
from app.db.user import create_user, get_user_by_email
from app.config import AUTH_SESSION_NAME
from app.dependencies import is_authorized, get_access_token, PWD_CONTEXT

from app.dependencies import is_authorized
from app.db.qa import (insert_question,
                       get_todays_qa_by_userid,
                       get_qa_histories,
                       delete_qa_by_id)
from app.db.qa_category import get_qa_categories
from llms.code_llama2 import CodeLlama2

In [3]:
landing_router = APIRouter()
landing_templates = Jinja2Templates(directory="app/templates/landing")

@landing_router.get("/", response_class=HTMLResponse)
async def profile(request: Request):
    """
    Renders the index.html template.

    Args:
        request (Request): The incoming request object.

    Returns:
        TemplateResponse: The rendered index.html template.
    """
    return landing_templates.TemplateResponse("index.html", {"request": request})


In [4]:
auth_router = APIRouter(prefix="/auth")
templates = Jinja2Templates(directory="app/templates")

def authenticate_user(email, password):
    """
    Authenticates a user by checking if the provided email and password match the stored credentials.

    Args:
        email (str): The email of the user.
        password (str): The password of the user.

    Returns:
        dict or bool: If the authentication is successful, returns the user dictionary. Otherwise, returns False.
    """
    user = get_user_by_email(email)
    if not user:
        return False
    if not PWD_CONTEXT.verify(password, user["password"]):
        return False
    user.pop('photo')
    return user


@auth_router.get("/login", response_class=HTMLResponse)
async def login_form(request: Request):
    """
    Handle the login request.

    Args:
        request (Request): The incoming request object.

    Returns:
        TemplateResponse: The response containing the login.html template.
    """
    return templates.TemplateResponse("login.html", {"request": request})


@auth_router.post("/login")
async def login(request: Request):
    """
    Handle the login request.

    Args:
        request (Request): The incoming request object.

    Returns:
        TemplateResponse: The response containing the login.html template.
    """
    try:
        form = await request.form()
        user_email = form.get("email", None)
        user_password = form.get("password", None)
        
        if not user_email or not user_password:
            return templates.TemplateResponse("login.html", {
                "request": request,
                "error": "Invalid username or password"
            })
        
        user = authenticate_user(user_email, user_password)
        if user is False:
            return templates.TemplateResponse("login.html", {
                "request": request,
                "error": "Invalid username or password"
            })
        request.session[AUTH_SESSION_NAME] = get_access_token({"sub": user})
        request.session["USER_ID"] = user.get('id')
        request.session["FULL_NAME"] = f"{user.get('first_name')} {user.get('last_name')}"
        return RedirectResponse("/dashboard/home", status_code=status.HTTP_302_FOUND)
    except Exception as e:
        print(e)
        return RedirectResponse("/auth/login?error=Something went wrong. Please try again later.", status_code=status.HTTP_302_FOUND)


@auth_router.get("/register", response_class=HTMLResponse)
async def register_form(request: Request):
    """
    Register a new user.

    Args:
        request (Request): The incoming request.

    Returns:
        TemplateResponse: The template response for the register page.
    """
    return templates.TemplateResponse("register.html", {"request": request})


@auth_router.post("/register", response_class=HTMLResponse)
async def register(request: Request):
    """
    Register a new user.

    Args:
        request (Request): The incoming request.

    Returns:
        TemplateResponse: The template response for the register page.
    """
    form = await request.form()
    first_name = form.get("first_name")
    last_name = form.get("last_name")
    email = form.get("email")
    password = form.get("password")
    password_repeat = form.get("password_repeat")

    if password != password_repeat:
        return templates.TemplateResponse("register.html", {
            "request": request, 
            "error": "Passwords do not match"
        })
    
    hashed_password = PWD_CONTEXT.hash(password)
    is_user_created = create_user(first_name, last_name, email, hashed_password)
    if is_user_created:
        return templates.TemplateResponse("register.html", {
            "request": request,
            "success": "Registration successful"
        })
    return templates.TemplateResponse("register.html", {
            "request": request,
            "error": "Registration failed"
        })


@auth_router.get("/forgot-password", response_class=HTMLResponse)
async def forgot_password(request: Request):
    """
    Handle the forgot password request.

    Args:
        request (Request): The incoming request object.

    Returns:
        TemplateResponse: The template response for the forgot password page.
    """
    return templates.TemplateResponse("forgot-password.html", {"request": request})


@auth_router.get("/logout", dependencies=[Depends(is_authorized)])
async def logout(request: Request):
    """
    Logs out the user and returns a message indicating successful logout.
    """
    request.session.pop(AUTH_SESSION_NAME, None)
    request.session.pop("USER_ID", None)
    request.session.pop("FULL_NAME", None)

    if request.session.get(AUTH_SESSION_NAME) is None:
        return RedirectResponse("/auth/login", status_code=status.HTTP_302_FOUND)

In [5]:
from datetime import datetime
from fastapi import APIRouter, Request, Depends, status
from fastapi.responses import HTMLResponse, RedirectResponse
from fastapi.templating import Jinja2Templates
from app.dependencies import is_authorized
from app.db.qa import (insert_question,
                       get_todays_qa_by_userid,
                       get_qa_histories,
                       delete_qa_by_id)
from app.db.qa_category import get_qa_categories
from llms.code_llama2 import CodeLlama2

chat_router = APIRouter(prefix="/chatbot")
chat_router.dependencies = [Depends(is_authorized)]
llm = CodeLlama2()


@chat_router.get("/qa", response_class=HTMLResponse)
async def chat_home(request: Request):
    """
    This function handles the chat home page request.

    Args:
        request (Request): The incoming request object.

    Returns:
        TemplateResponse: The template response containing the chatbot.html template, request object, and questions.
    """
    today_date = datetime.now().strftime("%Y-%m-%d")
    user_id = request.session["USER_ID"]
    categories = get_qa_categories()
    questions = get_todays_qa_by_userid(user_id, today_date)
    return templates.TemplateResponse(
        "chatbot.html", {
            "request": request,
            "categories": categories,
            "questions": questions,
            # "error": error
        })


@chat_router.post("/qa", response_class=HTMLResponse)
async def question_answer(request: Request):
    """
    Handles POST requests to the "/qa" route. This function takes a user's question, 
    validates the form data, and inserts the question into the database.

    Args:
        request (Request): The request object containing all the HTTP information.

    Returns:
        RedirectResponse: A redirection to "/chatbot/qa" if the question was successfully inserted.
        If there were any empty fields in the form or if the insertion failed, it redirects to 
        "/chatbot/qa" with an appropriate error message.
    """

    form = await request.form()
    empty_fields = [field.replace("_", " ") for field, value in form.items() if not value]
    
    if empty_fields:
        message = f"Empty fields: {', '.join(empty_fields)}"
        return RedirectResponse(f"/chatbot/qa?error={message}", status_code=status.HTTP_302_FOUND)
    
    user_id = request.session["USER_ID"]
    question_category = form.get("question_category")
    question = form.get("question")
    qa_response = llm.conversationa_chain_retrival(question)
    answer = qa_response["answer"]
    today_date = datetime.now().strftime("%Y-%m-%d %I:%M")
    insert_rsp = insert_question(user_id, question, answer, question_category, today_date)
    if insert_rsp:
        return RedirectResponse("/chatbot/qa", status_code=status.HTTP_302_FOUND)
    
    return RedirectResponse("/chatbot/qa?error=Something went wrong", status_code=status.HTTP_302_FOUND)


@chat_router.get("/history", response_class=HTMLResponse)
async def chat_history(request: Request):
    """
    This function handles the chat home page request.

    Args:
        request (Request): The incoming request object.

    Returns:
        TemplateResponse: The template response containing the chatbot.html template, request object, and questions.
    """
    user_id = request.session["USER_ID"]
    qa_histories = get_qa_histories(user_id)

    return templates.TemplateResponse(
        "all_qa.html", {
            "request": request,
            "qa_histories": qa_histories,
            # "error": error
        })


@chat_router.post("/history/delete", response_class=HTMLResponse)
async def chat_history_delte(request: Request):
    """
    Handle the login request.

    Args:
        request (Request): The incoming request object.

    Returns:
        TemplateResponse: The response containing the login.html template.
    """
    try:
        form = await request.form()
        qa_id = form.get("qa_id", None)
        
        if not qa_id:
            return RedirectResponse("/chatbot/history?error=Deletaion unsuccessful!", status_code=status.HTTP_302_FOUND)
        
        user_id = request.session["USER_ID"]
        response = delete_qa_by_id(qa_id, user_id)

        if response:
            return RedirectResponse("/chatbot/history?success=Deletaion successful!", status_code=status.HTTP_302_FOUND)
        return RedirectResponse("/chatbot/history?error=Something went wrong!", status_code=status.HTTP_302_FOUND)
    except Exception as e:
        print(e)
        return RedirectResponse("/auth/login?error=Something went wrong. Please try again later.", status_code=status.HTTP_302_FOUND)

In [7]:
import io
from base64 import b64encode
import imghdr
from fastapi import (APIRouter, Request, Depends, status,
                     UploadFile, File, HTTPException)
from fastapi.responses import HTMLResponse, RedirectResponse, StreamingResponse
from fastapi.templating import Jinja2Templates
from app.dependencies import is_authorized, PWD_CONTEXT
from app.db.user import (get_user_by_id, update_user, update_photo,
                         get_photo_by_user_id, update_password)
from app.db.address import (count_user_address, get_user_address_by_id,
                            update_user_address)


user_router = APIRouter(prefix="/user")
user_router.dependencies = [Depends(is_authorized)]


@user_router.get("/profile", response_class=HTMLResponse)
async def profile(request: Request):
    """
    Renders the profile.html template.

    Args:
        request (Request): The incoming request object.

    Returns:
        TemplateResponse: The rendered profile.html template.
    """
    user_id = request.session["USER_ID"]
    user = get_user_by_id(user_id)
    success = request.query_params.get("success", None)
    error = request.query_params.get("error", None)

    user_address = get_user_address_by_id(user_id)
    user["address"] = user_address
    if not user.get("photo"):
        user["profile_photo"] = None
    if user.get("photo"):
        user["profile_photo"] = b64encode(user.get("photo", "")).decode('utf-8')

    return templates.TemplateResponse(
        "profile.html", {
            "request": request,
            "user": user,
            "success": success,
            "error": error
        })


@user_router.post("/profile", response_class=HTMLResponse)
async def update_profile(request: Request):
    """
    Updates the user profile.

    Args:
        request (Request): The incoming request object.

    Returns:
        TemplateResponse: The rendered profile.html template.
    """
    form = await request.form()
    empty_fields = [field.replace("_", " ") for field, value in form.items() if not value]
    
    if empty_fields:
        message = f"Empty fields: {', '.join(empty_fields)}"
        return RedirectResponse(f"/user/profile?error={message}", status_code=status.HTTP_302_FOUND)
    
    first_name = form.get("first_name")
    last_name = form.get("last_name")
    email = form.get("email")
    update_rsp = update_user(
        request.session["USER_ID"],
        first_name,
        last_name,
        email
    )
    if update_rsp:
        request.session["FULL_NAME"] = f"{first_name} {last_name}"
        return RedirectResponse("/user/profile?success=Profile updated successfully", status_code=status.HTTP_302_FOUND)
    
    return RedirectResponse("/user/profile?error=Error updating profile", status_code=status.HTTP_302_FOUND)


@user_router.post("/address-update", response_class=HTMLResponse)
async def update_address(request: Request):
    """
    Updates the user profile.

    Args:
        request (Request): The incoming request object.

    Returns:
        TemplateResponse: The rendered profile.html template.
    """
    address = await request.form()
    empty_fields = [field.replace("_", " ") for field, value in address.items() if not value]
    
    if empty_fields:
        message = f"Empty fields: {', '.join(empty_fields)}"
        return RedirectResponse(f"/user/profile?error={message}", status_code=status.HTTP_302_FOUND)
    
    user_id = request.session["USER_ID"]
    user_address = get_user_address_by_id(user_id)
    address_id = user_address.get("id", count_user_address()+1)
    update_rsp = update_user_address(address_id, user_id, address)

    if update_rsp:
        return RedirectResponse("/user/profile?success=Profile updated successfully", status_code=status.HTTP_302_FOUND)
    return RedirectResponse("/user/profile?error=Error updating profile", status_code=status.HTTP_302_FOUND)


@user_router.get("/profile-photo")
async def display_image(request: Request):
    """
    Display the user's image based on the user ID.

    Args:
        request (Request): The incoming request object.

    Returns:
        StreamingResponse: The streaming response containing the user's image.

    Raises:
        HTTPException: If the user's image is not found.
    """
    user_id = request.session.get("USER_ID", None)
    user = get_photo_by_user_id(user_id)

    if user:
        return StreamingResponse(io.BytesIO(user.get("photo", None)), media_type="image/jpeg")
    raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Image not found")


@user_router.post("/photo-upload")
async def upload_file(request: Request, file: UploadFile = File()):
    """
    Uploads a file and updates the user's photo.

    Parameters:
        request (Request): The request object.
        file (UploadFile, optional): The file to be uploaded. Defaults to File().

    Returns:
        dict: A dictionary containing the filename and status of the upload.
    """
    file_content = await file.read()
    # Check if the uploaded file is a PNG or JPEG image
    file_type = imghdr.what(None, h=file_content)

    if file_type not in ["png", "jpeg", "jpg"]:
        return RedirectResponse("/user/profile?success=Supported file types are PNG/JPG/JPEG", status_code=status.HTTP_302_FOUND)

    user_id = request.session["USER_ID"]
    update_rsp = update_photo(user_id, file_content)
    if update_rsp:
        return RedirectResponse("/user/profile?success=Profile photo updated successfully", status_code=status.HTTP_302_FOUND)
    return RedirectResponse("/user/profile?error=Error updating profile photo", status_code=status.HTTP_302_FOUND)


@user_router.get("/settings", response_class=HTMLResponse)
def settings_form(request: Request):
    """
    Display the user's image based on the user ID.

    Args:
        request (Request): The incoming request object.

    Returns:
        StreamingResponse: The streaming response containing the user's image.

    Raises:
        HTTPException: If the user's image is not found.
    """
    success = request.query_params.get("success", None)
    error = request.query_params.get("error", None)
    return templates.TemplateResponse("settings.html", 
                                      {"request": request,
                                       "success": success,
                                       "error": error})


@user_router.post("/update-credentials", response_class=HTMLResponse)
async def update_credentials(request: Request):
    """
    Handle the settings page for a user.

    Args:
        request (Request): The incoming request object.

    Returns:
        RedirectResponse: The response object for redirecting to a different page.
    """
    try:
        form = await request.form()
        old_password = form.get("old_password")
        password = form.get("password")
        repeat_password = form.get("repeat_password")

        user_id = request.session["USER_ID"]
        user = get_user_by_id(user_id)
        if not user:
            return RedirectResponse("/user/settings?error=Current passwords didn't match", status_code=status.HTTP_302_FOUND)
        if not PWD_CONTEXT.verify(old_password, user["password"]):
            return RedirectResponse("/user/settings?error=Current passwords didn't match", status_code=status.HTTP_302_FOUND)

        if password != repeat_password:
            return RedirectResponse("/user/settings?error=New password and confirm password didn't match", status_code=status.HTTP_302_FOUND)
        
        hashed_password = PWD_CONTEXT.hash(password)
        update_rsp = update_password(user_id, hashed_password)
        if update_rsp:
            return RedirectResponse("/user/settings?success=Password updated successfully", status_code=status.HTTP_302_FOUND)
        return RedirectResponse("/user/settings?error=Password update failed", status_code=status.HTTP_302_FOUND)
    except Exception as e:
        print(e)
        return RedirectResponse("/user/settings?error=Something went wrong. Please try again later.", status_code=status.HTTP_302_FOUND)

In [8]:
from fastapi import APIRouter, Request, Depends
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from app.dependencies import is_authorized
from app.db.dashboardstats import get_question_stats, get_question_ratio
from app.config import MONTH_NAMES

dashboard_router = APIRouter(prefix="/dashboard")
dashboard_router.dependencies = [Depends(is_authorized)]


@dashboard_router.get("/home", response_class=HTMLResponse)
def chat_home(request: Request):
    """
    A function that handles the dashboard route.

    Returns:
        TemplateResponse: The response containing the rendered dashboard.html template.
    """
    user_id = request.session["USER_ID"]
    question_stats = question_stats_to_dict(user_id)
    question_ratio = question_stats_ratio(user_id)

    return templates.TemplateResponse(
        "dashboard.html", {
            "request": request,
            "question_stats": question_stats,
            "question_ratio": question_ratio
        })


def question_stats_to_dict(user_id):
    """
    Converts the question statistics from a list of tuples to a dictionary.

    Args:
        question_stats (list): A list of tuples containing the question statistics.

    Returns:
        dict: A dictionary containing the question statistics.
    """
    question_stats = get_question_stats(user_id)
    months = []
    frequency = []
    for question_stat in question_stats:
        months.append(MONTH_NAMES[question_stat[0]])
        frequency.append(question_stat[2])
    stats = {
        "months": months,
        "frequency": frequency
    }
    return stats


def question_stats_ratio(user_id):
    """
    Converts the question statistics from a list of tuples to a dictionary.

    Args:
        question_stats (list): A list of tuples containing the question statistics.

    Returns:
        dict: A dictionary containing the question statistics.
    """
    question_stats = get_question_ratio(user_id)
    if not question_stats:
        return {"python": 0, "sql": 0}
    question_stats = {key: value for key, value in question_stats}
    
    return question_stats


In [9]:
from fastapi import Depends, FastAPI
from fastapi.staticfiles import StaticFiles
from starlette.middleware.sessions import SessionMiddleware
from app.config import SECRET_KEY

app = FastAPI()
app.add_middleware(SessionMiddleware, secret_key=SECRET_KEY)
app.mount("/assets", StaticFiles(directory="app/templates/assets"), name="assets")
app.include_router(landing_router)
app.include_router(auth_router)
app.include_router(chat_router)
app.include_router(user_router)
app.include_router(dashboard_router)

In [ ]:
import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app, host="localhost", port=8000)
    server = uvicorn.Server(config)
    await server.serve()

# !pip install nest_asyncio
# import nest_asyncio
# import uvicorn

# if __name__ == "__main__":
#     nest_asyncio.apply()
#     uvicorn.run(app)

# poetry run uvicorn main:app --host localhost --reload


INFO:     Started server process [87125]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)


INFO:     ::1:53980 - "GET / HTTP/1.1" 200 OK
INFO:     ::1:53980 - "GET /assets/fonts/S6uyw4BMUTPHjx4wXiWtFCc.woff2 HTTP/1.1" 404 Not Found
INFO:     ::1:53981 - "GET /assets/fonts/S6u9w4BMUTPHh6UVSwiPGQ3q5d0.woff2 HTTP/1.1" 404 Not Found
INFO:     ::1:53982 - "GET /assets/fonts/S6u9w4BMUTPHh7USSwiPGQ3q5d0.woff2 HTTP/1.1" 404 Not Found
INFO:     ::1:53982 - "GET /dashboard/home HTTP/1.1" 200 OK
INFO:     ::1:53982 - "GET /user/profile-photo HTTP/1.1" 200 OK
INFO:     ::1:53982 - "GET /chatbot/qa HTTP/1.1" 200 OK
INFO:     ::1:53982 - "GET /user/profile-photo HTTP/1.1" 200 OK
INFO:     ::1:53982 - "GET /chatbot/qa HTTP/1.1" 200 OK
INFO:     ::1:53982 - "GET /user/profile-photo HTTP/1.1" 200 OK
INFO:     ::1:53982 - "GET /chatbot/history HTTP/1.1" 200 OK
INFO:     ::1:53982 - "GET /user/profile-photo HTTP/1.1" 200 OK
INFO:     ::1:53982 - "GET /dashboard/home HTTP/1.1" 200 OK
INFO:     ::1:53982 - "GET /user/profile-photo HTTP/1.1" 200 OK
INFO:     ::1:53982 - "GET /chatbot/qa HTTP/1.1"

In [ ]:
# lsof -i tcp:8000
# kill -9 PID